In [1]:
import os
import pathlib
from pathlib import Path
import glob
import re
import cx_Oracle as oracle
import pandas as pd
import openpyxl
import shutil

In [2]:
#カレントディレクトリを取得
cwd = os.getcwd()
    
#カレントディレクトリ内の対象フォルダ名を取得(複数ファイル一括)
target_folders = []#アップ用
target_folders1 = []#解除用
for f in glob.glob(cwd  + "\*"):
    folder_name = os.path.split(f)[1]
    
    if re.findall("★登録済_40A_品連NG", folder_name):
        target_folders1.append(folder_name)
    elif re.findall("40A_品連NG", folder_name):
        target_folders.append(folder_name)

            
#対象ディレクトリのパスを作成
path_list = [cwd + "\\" + target_folder for target_folder in target_folders]#アップ用
path_list1 = [cwd + "\\" + target_folder for target_folder in target_folders1]#解除用
    
#データフォルダ名のリスト作成
folder_name_list = [target_folder for target_folder in target_folders]#アップ用
folder_name_list1 = [target_folder for target_folder in target_folders1]#解除用

### MESサーバーへの接続

In [4]:
#CX oracleでMESサーバーへ接続（接続情報以外は定型）
class conn_MES_LWR:
    def __init__(self, host= "10.60.28.37", port="1521", service="psh3dbv",
                       scheme="tabuser",username="tabuser",password="tab123"):        
        self.host = host
        self.port = port
        self.service  = service

        self.scheme   = scheme
        self.username = username
        self.password = password
    
    def __enter__(self):
        
        # tns:Oracleが命名したDB接続用インターフェース技術の名前
        
        # インターフェイスオブジェクトの作成
        self.tns  = oracle.makedsn(self.host, self.port, service_name=self.service) if self.host else None
        # 接続を確立
        self.conn = oracle.connect(self.username, self.password, self.tns) if self.tns else None
        # カーソルの取得
        self.curs = self.conn.cursor() if self.conn else None
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        if self.curs is not None: self.curs.close()
        if self.conn is not None: self.conn.close()

### DBにアップする用のSQL

In [5]:
#保留セル用SQL
insert_sample = 'INSERT INTO "TABUSER"."HoldingCellList(QCCont)40A" ("CellId", "InstructReason", "InstructNo", "InstructCont", "RecissionReason", "RecissionNo", "RecissionCont") VALUES (:1,:2,:3,:4,:5,:6,:7)'

#保留解除用SQL
update_sample = 'UPDATE "TABUSER"."HoldingCellList(QCCont)40A" set "RecissionReason" = :5  , "RecissionNo" = :6 , "RecissionCont" = :7 Where "CellId" = :1 AND "InstructNo" = :2'

### DBにアップする関数の定義

In [6]:
#保留セルをDBにアップロードする関数
def upload_data(insert_sql=None, df=None):
       
    with conn_MES_LWR() as mesdb:
        # executemany()で複数行のデータを一括でインサート
        mesdb.curs.executemany(insert_sql,df)
        mesdb.conn.commit()

In [7]:
#保留解除セルをDBにアップデートする関数
def update_data(update_sql=None, df=None):
    
    with conn_MES_LWR() as mesdb:
        # executemany()はupdate関数が使用不可の為、削除
        mesdb.curs.execute(update_sql,df)
        mesdb.conn.commit()

### 保留セルをDBにアップする

In [8]:
#エクセルファイルを読み込みdf化
sample_dfs = pd.DataFrame(columns = [])
for i in folder_name_list:
    tmp = pd.read_excel(i,sheet_name="Sheet1",header=0)
    sample_dfs = pd.concat([sample_dfs,tmp])

In [10]:
# NaNがあると何かと不都合なので、NaNを空文字に変換
sample_dfs = sample_dfs.fillna("")

In [11]:
 # DBに格納できるようにデータフレームを二次元配列に変換
sample_dfs = sample_dfs.values

In [12]:
#DBへアップロード
try :
    upload_data(insert_sample,list(sample_dfs))

except Exception:
    print('新規データはありません')

### 保留セルの解除内容をDBにアップする

In [13]:
#エクセルファイルを読み込みdf化
sample_dfs1 = pd.DataFrame(columns = [])
for i in folder_name_list1:
    tmp = pd.read_excel(i,sheet_name="Sheet1",header=0)
    sample_dfs1 = pd.concat([sample_dfs1,tmp])

In [14]:
#保留解除セルのデータを加工しDBへアップロード
try :
    sample_dfs2 = sample_dfs1[['解除_理由','解除_品連番号','解除_指示内容','セルID','品連番号']]
    sample_dfs2.dropna(inplace=True)
    sample_dfs2=sample_dfs2.values
    
    #DBへアップロード
    for sample_df in sample_dfs2:
        update_data(update_sample,sample_df)
    

except Exception:
    print('保留解除データはありません')

C:\Users\E283674\AppData\Local\Temp\ipykernel_1060\847251986.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_dfs2.dropna(inplace=True)


### 読み込んだExcelファイルを移動する

In [15]:
#登録を実施したファイル
#登録したファイルの名前を変更する

name_list=[]
path=[]
for i in folder_name_list:
    os.rename(i,'★登録済_'+i)
    name_list.append('★登録済_'+i)
    path.append(cwd + "\\" + '★登録済_'+i)

In [16]:
#移動先ファイルの指定
new_file_path=cwd+"\\"+'登録済み'

#使用したファイルを登録済みファイルへ移動 
for file in path:
    shutil.move(file,new_file_path)

In [17]:
#保留解除実施ファイル
#ファイル名を★登録済から★解除済へ変更
name=[]
name_list1=[]
path1=[]

for l in folder_name_list1:
    name.append(l.lstrip('★登録済_'))

for i,r in zip(folder_name_list1 , name):
    os.rename(i,'★解除済_'+r)
    name_list1.append('★解除済_'+r)
    path1.append(cwd + "\\" + '★解除済_'+r)

In [18]:
#移動先ファイルの指定
new_file_path1=cwd+"\\"+'保留解除済み'

#使用したファイルを解除済みファイルへ移動 
for file1 in path1:
    shutil.move(file1,new_file_path1)

In [38]:
# コンソールがすぐに消えてしまわないようにするための処理
input("処理を完了するには、任意のキーを押してください。")

処理を完了するには、任意のキーを押してください。 


''